# Загрузка датасета и первый взгляд

In [1]:
!mkdir data
!wget https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/hometask/sem8/zhestov.ea%40phystech.edu.csv -O data/zhestov.ea@phystech.edu.csv

--2021-04-05 21:53:44--  https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/hometask/sem8/zhestov.ea%40phystech.edu.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 280043 (273K) [text/plain]
Saving to: ‘data/zhestov.ea@phystech.edu.csv’

data/zhestov.ea@phy 100%[===================>] 273.48K  --.-KB/s    in 0.03s   

2021-04-05 21:53:44 (8.54 MB/s) - ‘data/zhestov.ea@phystech.edu.csv’ saved [280043/280043]



In [2]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
data = pd.read_csv('data/zhestov.ea@phystech.edu.csv', header=None)

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,-1.761398,-0.156257,-0.005979,0.860942,0.091872,-0.331193,0.749937,0.019989,0.032373,0.319947
1,0.0,-1.303103,0.387844,-1.128526,1.457206,0.812469,-0.022561,-2.500210,-1.444416,-0.235715,0.925951
2,0.0,-0.249461,1.443965,0.620882,0.470192,0.988734,0.028974,-0.640343,-0.405336,0.084876,0.031681
3,0.0,-0.230755,-0.765686,1.712833,-0.874092,0.423669,-1.889826,0.014605,-0.661475,0.213601,1.189151
4,0.0,0.526141,1.154553,-1.136941,-0.689502,0.684815,0.901370,-1.486596,-0.721248,0.468192,2.423638


In [5]:
column_names = list(map(lambda num: f"X{num - 1}", data.columns))
column_names[0] = 'Y'
column_names
data.columns = column_names
data.describe()

,Y,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.474000,0.072231,-0.051437,-0.060313,0.032660,0.006941,-0.024947,0.006426,-0.028679,-0.020293,-0.026283
std,0.499573,1.016500,0.998119,0.983231,1.005172,0.993745,0.991906,0.998349,0.961597,1.003385,0.991686
min,0.000000,-3.130053,-3.287880,-2.842902,-2.870782,-2.696203,-3.949313,-2.933166,-3.370755,-3.420011,-3.275969
25%,0.000000,-0.622599,-0.753558,-0.709180,-0.669469,-0.667142,-0.732571,-0.644894,-0.649648,-0.690803,-0.688340
50%,0.000000,0.071710,-0.058918,-0.022007,0.035155,0.002624,-0.052348,0.013527,0.001218,-0.042588,-0.005404
75%,1.000000,0.775350,0.649182,0.604349,0.697111,0.669500,0.693689,0.745126,0.625735,0.671355,0.647797
max,1.000000,3.058372,3.327818,2.980125,3.318213,3.409064,3.167181,2.965234,3.022946,2.980623,3.057821


In [6]:
X = data.drop('Y', axis=1)
y = data['Y']
print(X.shape, y.shape)

(1000, 10) (1000,)


In [ ]:
sns.pairplot(data)

Я бы использовал тут `Probit`, ибо данные явно нормальные, но логит так логит, чё нам

# Задание
## Вопрос 1
Постройте модель логистической регрессии $Y$ от $X$ и свободного коэффициента. Каково отношение шансов $Y=1$ к событию $f(X) = 1$, где $f(X)$ - отклик?

In [7]:
logit_model = sm.Logit(endog=y, exog=sm.add_constant(X)).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.341465
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      Y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      989
Method:                           MLE   Df Model:                           10
Date:                Mon, 05 Apr 2021   Pseudo R-squ.:                  0.5064
Time:                        21:53:47   Log-Likelihood:                -341.46
converged:                       True   LL-Null:                       -691.79
Covariance Type:            nonrobust   LLR p-value:                4.534e-144
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1498      0.098     -1.524      0.127      -0.342       0.043
X0            -0.1269      0.096     -1.326      0.185      -0.315       0.061
X1             0.0071      0.096      0.074      0.941      -0.182       0.196
X2             0.0358      0.097      0.370      0.711      -0.154       0.225
X3            -0.0106      0.095     -0.111      0.911      -0.197       0.176
X4             0.0638      0.096      0.663      0.507      -0.125       0.252
X5             1.4756      0.129     11.421      0.000       1.222       1.729
X6             1.4502      0.126     11.554      0.000       1.204       1.696
X7             1.2688      0.125     10.115      0.000       1.023       1.515
X8             1.4103      0.124     11.396      0.000       1.168       1.653
X9             1.5051      0.128     11.740      0.000       1.254       1.756
==============================================================================
"""

Видимо тут хотят одну дробь делить на другую дробь, что странновато, но ладно.

In [8]:
y.sum()

474.0

In [9]:
import numpy as np
np.exp(logit_model.params)

const    0.860893
X0       0.880782
X1       1.007126
X2       1.036445
X3       0.989473
X4       1.065835
X5       4.373517
X6       4.264158
X7       3.556430
X8       4.097173
X9       4.504535
dtype: float64

Андрей сказал, что тут от нас требуют штуку под названием *Odds ratio*

In [10]:
p_endog = y.sum()
p_endog_normed = p_endog / len(y)
f = logit_model.predict(sm.add_constant(X))
response = list(map(round, f))
p_resp = len(list(filter(lambda x: x == 1, response)))
p_resp_normed = p_resp / len(response)
OR = (p_endog_normed / (1 - p_endog_normed)) * ((1 - p_resp_normed) / p_resp_normed)

In [11]:
np.round(OR, decimals=2)

1.0

## Вопрос 2
Какова точность (Accuracy) классификации модели при пороге вероятности для классификации $p_0 = 0.5$?

In [12]:
np.round(accuracy_score(y, response), decimals=2)

0.84

## Вопрос 3
Постройте модель логистической регрессии $Y$ от $X$ и свободного коэффициента для первых $200$ элементов выборки. Имеет ли смысл данная модель согласно критерию Вальда? (p-value$=0.05$)

In [13]:
X = X[0:200]
y = y[0:200]

In [14]:
new_model = sm.Logit(endog=y, exog=sm.add_constant(X)).fit()
new_model.summary()

Optimization terminated successfully.
         Current function value: 0.284468
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      Y   No. Observations:                  200
Model:                          Logit   Df Residuals:                      189
Method:                           MLE   Df Model:                           10
Date:                Mon, 05 Apr 2021   Pseudo R-squ.:                  0.5837
Time:                        21:53:53   Log-Likelihood:                -56.894
converged:                       True   LL-Null:                       -136.66
Covariance Type:            nonrobust   LLR p-value:                 4.034e-29
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1263      0.243     -0.520      0.603      -0.602       0.349
X0            -0.5263      0.269     -1.955      0.051      -1.054       0.001
X1            -0.0702      0.257     -0.274      0.784      -0.573       0.433
X2            -0.1332      0.288     -0.463      0.643      -0.697       0.430
X3             0.6546      0.310      2.113      0.035       0.047       1.262
X4            -0.5482      0.260     -2.106      0.035      -1.058      -0.038
X5             2.1418      0.408      5.247      0.000       1.342       2.942
X6             2.0313      0.367      5.531      0.000       1.312       2.751
X7             2.0748      0.418      4.969      0.000       1.257       2.893
X8             1.8166      0.358      5.080      0.000       1.116       2.518
X9             1.8886      0.348      5.420      0.000       1.206       2.572
==============================================================================
"""

In [15]:
from functools import reduce

string = reduce(lambda x, a: a + x, reversed(' = '.join(map(lambda x: f'X{x}', range(10)))))
string = 'const = ' + string

In [16]:
new_model.wald_test(string).summary()

'<Wald test (chi2): statistic=[[42.78012277]], p-value=5.4439283143136035e-06, df_denom=10>'

Модель имеет смысл, т.к. нулевая гипотеза о том, что все переменные = 0 имеет pvalue меньше 0.05

## Вопрос 4
Можно ли обнулить первые четыре переменные согласно критерию Вальда? (p-value$=0.05$)

In [17]:
string = reduce(lambda x, a: a + x, reversed(' = '.join(map(lambda x: f'X{x}', range(4)))))

In [18]:
new_model.wald_test(string).summary()

'<Wald test (chi2): statistic=[[7.28979603]], p-value=0.0632127347911899, df_denom=3>'

Нельзя обнулить, ибо нулевая гипотеза, что первые 4 переменные равны нулю, имеет pvalue больше $0.05$